# IMPORTS

In [ ]:
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

In [ ]:
path = "Indian-monuments//images"
for dirpath, dirnames, filenames in os.walk(path):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
train_dir =  path + "/train/"
test_dir = path + "/test/"
train_dir

In [ ]:
test_dir

In [ ]:
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names

In [ ]:
def view_random_image(target_dir, target_class):
    target_folder = target_dir  + target_class
    random_image = random.sample(os.listdir(target_folder), 1)
    img = mpimg.imread(target_folder + "/" + random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off");
    print(f"Image shape: {img.shape}") 
    return img

In [ ]:
import random
img = view_random_image(target_dir=train_dir, target_class=random.choice(class_names))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(300, 300),
                                               batch_size=32,
                                               class_mode='categorical') 


test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=(300, 300),
                                              batch_size=32,
                                              class_mode='categorical')

In [ ]:
model_10 = Sequential([
  Conv2D(10, 3, activation='relu', input_shape=(300, 300, 3)),
  MaxPool2D(),
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(24, activation='softmax')
])

In [ ]:
model_10.compile(loss='categorical_crossentropy',
                 optimizer=tf.keras.optimizers.Adam(),
                 metrics=['accuracy'])

history_10 = model_10.fit(train_data,
                          epochs=5,
                          steps_per_epoch=len(train_data),
                          validation_data=test_data,
                          validation_steps=len(test_data))

In [ ]:
def load_and_prep_image(filename, img_shape=300):
  img = tf.io.read_file(filename)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.resize(img, size = [img_shape, img_shape])
  img = img/255.    
  return img


def pred_and_plot(model, filename, class_names):
  img = load_and_prep_image(filename)
  pred = model.predict(tf.expand_dims(img, axis=0))
  if len(pred[0]) > 1: 
    pred_class = class_names[pred.argmax()]
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] 
  plt.imshow(img)
  plt.title(f"Prediction made: {pred_class}")
  plt.axis(False);

In [ ]:

pred_and_plot(model_10, "Indian-monuments//images//test//tanjavur temple//20.jpeg", class_names)

In [ ]:
pred_and_plot(model_10, "Indian-monuments//images//test//Sun Temple Konark//14.jpg", class_names)

In [ ]:
pred_and_plot(model_10, "Indian-monuments//images//test//tajmahal//92.jpg", class_names)    

In [ ]:
pred_and_plot(model_10,"Indian-monuments//images//test//Bgscet//test3.jpg",class_names)

In [ ]:
model_10.save('my_combined_model.h5')